In [2]:
import requests
from bs4 import BeautifulSoup



In [2]:
res = requests.get(link)

soup = BeautifulSoup(res.text, 'html.parser')

In [27]:
data=[]

for sp in soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):
    
    book_title=sp.find_all('a')[-1].get('title')
    book_link="https://books.toscrape.com/catalogue/"+sp.find_all('a')[-1].get('href')
    image_link= 'https://books.toscrape.com/'+sp.find('img').get('src')
    rating=sp.find('p').get('class')[-1]
    price=sp.find('p',class_="price_color").text[1:]
    availability=sp.find('p',class_="instock availability").text.strip()

    
    data.append([book_title,price,rating,availability,book_link,image_link])
    

In [30]:
len(data)

20

In [4]:
data=[]

for page in range(1,51):
    

    link = 'https://books.toscrape.com/catalogue/page-'+str(page)+'.html'
    res = requests.get(link)

    soup = BeautifulSoup(res.text, 'html.parser')

    
    

    for sp in soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):
    
        book_title=sp.find_all('a')[-1].get('title')
        book_link="https://books.toscrape.com/catalogue/"+sp.find_all('a')[-1].get('href')
        image_link= 'https://books.toscrape.com/'+sp.find('img').get('src')
        rating=sp.find('p').get('class')[-1]
        price=sp.find('p',class_="price_color").text[1:]
        availability=sp.find('p',class_="instock availability").text.strip()

    
        data.append([book_title,price,rating,availability,book_link,image_link])
    


KeyboardInterrupt



In [35]:
len(data)

420

In [39]:
data[419]

['Off the Hook (Fishing for Trouble #1)',
 '£47.67',
 'Three',
 'In stock',
 'https://books.toscrape.com/catalogue/off-the-hook-fishing-for-trouble-1_601/index.html',
 'https://books.toscrape.com/../media/cache/1d/78/1d78fe226e1adb9cb591fa21f8a9bf68.jpg']

In [5]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

data = []

for i in tqdm(range(1,51)):        # Going through each page one by one

    link = 'https://books.toscrape.com/catalogue/page-' + str(i) + '.html'     # Creating link for each page

    res = requests.get(link)                                                   # Sending Request to the link
    soup = BeautifulSoup(res.text, 'html.parser')                              # Creating a soup for that page
    
    
    for sp in soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):
    
        img_link   = 'https://books.toscrape.com/' + sp.find('img').get('src')[3:]
        book_link  = 'https://books.toscrape.com/catalogue/' + sp.find_all('a')[-1].get('href')
        title      = sp.find_all('a')[-1].get('title')
        rating     = sp.find('p').get('class')[-1]
        price      = sp.find('p',class_ = 'price_color').text[2:]
        stock      = sp.find('p', class_ = 'instock availability').text.strip()

        data.append([title, rating, price, stock, book_link, img_link])        # Saving scraped data in a list


df = pd.DataFrame(data, columns = ['title', 'rating', 'price', 'stock', 'book_link','img_link'])
df.to_csv('books.csv', index = False)


100%|██████████| 50/50 [01:06<00:00,  1.33s/it]


NOW, EXTRACTING DETAILS OF EACH BOOKM ALSO FROM YTHEIR LINK IN CSV FILE

In [6]:
data = []

for link in tqdm(df['book_link']):
    
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    typ = soup.find('ul', class_ = 'breadcrumb').find_all('a')[2].text
    
    temp = soup.find('table', class_ = 'table table-striped').find_all('td')
    
    upc      = temp[0].text
    price_x  = temp[2].text[2:]
    price_i  = temp[3].text[2:]
    tax      = temp[4].text[2:]
    qn       = temp[5].text
    reviews  = temp[6].text
    
    data.append([typ, price_x, price_i, tax, qn, upc, reviews])

100%|██████████| 1000/1000 [23:36<00:00,  1.42s/it] 


In [7]:
df = pd.DataFrame(data, columns = ['category','price_e_tax', 'price_i_tax', 'tax', 'quantity', 'upc', 'reviews'])
df.to_csv('data.csv', index = False)


COMBINING DATASETS


In [8]:
df_1 = pd.read_csv('books.csv')
df_2 = pd.read_csv('data.csv')

df = pd.DataFrame()

df['title']         = df_1['title']
df['upc']           = df_2['upc']
df['category']      = df_2['category']
df['price_e_tax']   = df_2['price_e_tax']
df['price_i_tax']   = df_2['price_i_tax']
df['tax']           = df_2['tax']
df['rating']        = df_1['rating']
df['reviews']       = df_2['reviews']
df['quantity']      = df_2['quantity']
df['stock']         = df_1['stock']

df['book_link']     = df_1['book_link']
df['img_link']      = df_1['img_link']



In [9]:
df.to_csv('final.csv', index = False)